In [1]:
import time

import numpy as np
from numba import njit, prange
from SALib.sample import saltelli
from SALib.analyze import sobol

Зафиксируйте какую-либо многомерную скалярную функцию
и реализуйте для неё анализ чувствительности по методу Соболя
на Python с использованием библиотеки SALib.

In [2]:
a = 0.03
b = 0.8

def model(x):
    return a * np.sin(x[0]) ** 2 + b * np.cos(x[1]) * np.sin(x[1]) + a * x[0] * np.sin(x[2]) ** 3

In [3]:
problem = {
    'num_vars' : 3,
    'names' : ['x1', 'x2', 'x3'],
    'bounds' : [[-np.pi, np.pi],
                [-np.pi, np.pi],
                [-np.pi, np.pi]
                ]
}

1) Проведите анализ чувствительности, проверьте сходимость, измерьте тайминги, 25 баллов

In [4]:
n = 160_000
start = time.time()
param_values = saltelli.sample(problem, n)
print("samples generation took %s seconds" %(time.time() - start))
Y = np.zeros([param_values.shape[0]])

start = time.time()
for i, X in enumerate(param_values):
    Y[i] = model(X)

print("model evaluation took %s seconds" %(time.time() - start))

start = time.time()
Si = sobol.analyze(problem, Y)

print("SA took %s seconds" %(time.time() - start))
print(Si['S1'])
print(Si['ST'])
print("x1-x2:", Si['S2'][0,1])
print("x1-x3:", Si['S2'][0,2])
print("x2-x3:", Si['S2'][1,2])

/tmp/ipykernel_873532/579809499.py:3: DeprecationWarning: `salib.sample.saltelli` will be removed in SALib 1.5. Please use `salib.sample.sobol`
  param_values = saltelli.sample(problem, n)
/home/kirill/anaconda3/lib/python3.9/site-packages/SALib/sample/saltelli.py:109: UserWarning: 
        Convergence properties of the Sobol' sequence is only valid if
        `N` (160000) is equal to `2^n`.
        
  warnings.warn(msg)


samples generation took 7.364480018615723 seconds
model evaluation took 7.5197014808654785 seconds
SA took 9.481085062026978 seconds
[1.38562395e-03 9.87190766e-01 1.00462815e-05]
[0.01280634 0.9871912  0.0114178 ]
x1-x2: 1.2735497044280564e-05
x1-x3: 0.011417271624314943
x2-x3: -1.1573953172539859e-05


Индексы чувствительности первого порядка положительные, но x2-x3 остается отрицательным, я пробовал увеличивать количество итераций, но это не меняет ситуацию и > 1_000_000 итераций я уже не могу запускать, не хватает памяти 

2) Ускорьте вычисления Python с использованием любой из имеющихся возможностей (PyBind11, ctypes, cython, numba), 25 баллов

In [5]:
@njit
def model_with_njit(x):
    return a * np.sin(x[0]) ** 2 + b * np.cos(x[1]) * np.sin(x[1]) + a * x[0] * np.sin(x[2]) ** 3

In [6]:
n = 80000
start = time.time()
param_values = saltelli.sample(problem, n)
print("without optimizations")

print("samples generation took %s seconds" %(time.time() - start))
Y = np.zeros([param_values.shape[0]])

start = time.time()
for i, X in enumerate(param_values):
    Y[i] = model(X)

print("model evaluation took %s seconds" %(time.time() - start))

start = time.time()
Si = sobol.analyze(problem, Y)

print("SA took %s seconds" %(time.time() - start))

print("optimized with njit")
print("samples generation took %s seconds" %(time.time() - start))
Y = np.zeros([param_values.shape[0]])

start = time.time()
for i, X in enumerate(param_values):
    Y[i] = model_with_njit(X)

print("model evaluation took %s seconds" %(time.time() - start))

start = time.time()
Si = sobol.analyze(problem, Y)

print("SA took %s seconds" %(time.time() - start))

/tmp/ipykernel_873532/1662838087.py:3: DeprecationWarning: `salib.sample.saltelli` will be removed in SALib 1.5. Please use `salib.sample.sobol`
  param_values = saltelli.sample(problem, n)
/home/kirill/anaconda3/lib/python3.9/site-packages/SALib/sample/saltelli.py:109: UserWarning: 
        Convergence properties of the Sobol' sequence is only valid if
        `N` (80000) is equal to `2^n`.
        
  warnings.warn(msg)


without optimizations
samples generation took 3.606050729751587 seconds
model evaluation took 3.8315343856811523 seconds
SA took 4.32394814491272 seconds
optimized with njit
samples generation took 4.324074745178223 seconds
model evaluation took 0.5789656639099121 seconds
SA took 4.122646331787109 seconds


С использованием jit заметно существенное ускорение

3) Попробуйте добавить параллелизм в вычисления, 25 баллов

In [7]:
n = 80000

start = time.time()
param_values = saltelli.sample(problem, n)

print("with prange")
print("samples generation took %s seconds" %(time.time() - start))
Y = np.zeros([param_values.shape[0]])

start = time.time()
for i in prange(n):
    Y[i] = model(param_values[i])

print("model evaluation took %s seconds" %(time.time() - start))

start = time.time()
Si = sobol.analyze(problem, Y)

print("SA took %s seconds" %(time.time() - start))



start = time.time()
param_values = saltelli.sample(problem, n)

print("with njit and prange")
print("samples generation took %s seconds" %(time.time() - start))
Y = np.zeros([param_values.shape[0]])

start = time.time()
for i in prange(n):
    Y[i] = model_with_njit(param_values[i])

print("model evaluation took %s seconds" %(time.time() - start))

start = time.time()
Si = sobol.analyze(problem, Y)

print("SA took %s seconds" %(time.time() - start))

/tmp/ipykernel_873532/1697358274.py:4: DeprecationWarning: `salib.sample.saltelli` will be removed in SALib 1.5. Please use `salib.sample.sobol`
  param_values = saltelli.sample(problem, n)
/home/kirill/anaconda3/lib/python3.9/site-packages/SALib/sample/saltelli.py:109: UserWarning: 
        Convergence properties of the Sobol' sequence is only valid if
        `N` (80000) is equal to `2^n`.
        
  warnings.warn(msg)


with prange
samples generation took 3.668898820877075 seconds
model evaluation took 0.48519110679626465 seconds
SA took 4.518218755722046 seconds


/tmp/ipykernel_873532/1697358274.py:24: DeprecationWarning: `salib.sample.saltelli` will be removed in SALib 1.5. Please use `salib.sample.sobol`
  param_values = saltelli.sample(problem, n)


with njit and prange
samples generation took 3.610607147216797 seconds
model evaluation took 0.04232215881347656 seconds
SA took 4.038254261016846 seconds


При использовании prange стало еще быстрее, как в случае с njit так и без него 
